In [1]:
import chromadb
from chromadb.api.types import EmbeddingFunction, QueryResult


class Chroma:
    def __init__(
        self,
        embedding: EmbeddingFunction,
        collection_name: str,
        path: str = './vector_db_bge_3'
    ) -> None:
        """
        Инициализирует класс для работы с ChromaDB. Используется для первоначальной загрузки данных в БД.
        Args:
            embedding (EmbeddingFunction): функция эмбеддинга.
            collection_name (str): название коллекции в ChromaDB.

        """
        self.embedding = embedding
        self.collection_name = collection_name
        self.chroma_client = chromadb.PersistentClient(path=path)

        self.collection = self.chroma_client.get_or_create_collection(
            name=self.collection_name,
            embedding_function=self.embedding,
            metadata={"hnsw:space": "cosine"} # доступны по дефолту l2 (Squared L2), Inner product (ip), cosine (Cosine similarity)  
        )

    def add_docs(self, data_dict: dict) -> None:
        """
        Adds documents to the ChromaDB.
        
        Args:
            data_dict (dict): Dictionary representing the documents. 
                                Must contain 'docs', cls_1, cls_2, 'sources', 'questions', and 'ids' fields.
        """
        docs = data_dict.get("docs")
        cls_1 = data_dict.get("cls_1")
        cls_2 = data_dict.get("cls_1")
        sources = data_dict.get("sources")
        ids = data_dict.get("ids")

        if not all([docs, sources, ids, cls_1, cls_2]):
            raise ValueError("data_dict must contain 'docs', 'cls_1', 'cls_2', 'sources' and 'ids' fields.")

        metadatas = [
            {"source": sources[i],
             "cls_1": cls_1[i],
             "cls_2": cls_2[i]} for i in range(len(docs))
        ]

        # Batch processing, чтобы все доки за один раз не обрабатывать 
        for start in range(0, len(docs), 100):
            end = start + 100
            self.collection.add(
                documents=docs[start:end],
                metadatas=metadatas[start:end],
                ids=ids[start:end],
            )

    def get_relevant_docs(self, user_query: str):
        """
        Получает документы из ChromaDB, удовлетворяющие пользовательскому запросу.
        Args:
            query (str): пользовательский запрос.
            collection_name (str): название коллекции ChromaDB, где находятся документы.
            embedding_info (dict): информация об эмбеддинге.
        Returns:
            Объект QueryResult с аттрибутами metadatas, documents и distances.
        """
        # user_query = self.embedding(user_query)
                
        if self.collection.count() > 0:
            return self.collection.query(
                query_texts=user_query,
                include=["metadatas", "documents", "distances"],
                n_results=5,
            )
        else:
            return collection.query(
                query_texts=query,
                include=["metadatas", "documents", "distances"],
                n_results=1,
            )

In [ ]:
import chromadb.utils.embedding_functions as embedding_functions
embedding = embedding_functions.SentenceTransformerEmbeddingFunction(model_name="deepvk/USER-bge-m3")

df['ids'] = [f"id_{i}" for i in range(len(df))]
df['sources'] = [f"link_{i}" for i in range(len(df))]

test_data = {
    "docs": df["concat"].tolist(),
    "cls_1": df["Классификатор 1 уровня"].tolist(),,
    "cls_2": df["Классификатор 2 уровня"].tolist(),,
    "sources": df["sources"].tolist(),
    "ids": df["ids"].tolist()
}

a = Chroma(
    embedding = embedding, 
    collection_name='bge_concated_3'
)
a.add_docs(test_data)

In [ ]:
def get_docs(query, threshold=0.7):
    result = a.get_relevant_docs(user_query=row['Вопрос пользователя'])
    docs = {
        "metadatas": [meta for meta, dist in zip(data["metadatas"], data["distances"]) if dist < threshold],
        "documents": [doc for doc, dist in zip(data["documents"], data["distances"]) if dist < threshold],
    }
    return docs

In [ ]:
import os
from dotenv import load_dotenv
from openai import OpenAI
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
client = OpenAI(
    api_key=OPENAI_API_KEY,
    base_url="https://api.proxyapi.ru/openai/v1",
)

In [ ]:
class LLM:
    OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

    def __init__(self, model, system_prompt):
        self.model = model
        self.system_prompt = system_prompt
        self.client = OpenAI(api_key=OPENAI_API_KEY, base_url="https://api.proxyapi.ru/openai/v1")
        self.messages = [{'role': 'system', 'content': system_prompt}]
    
    def chat(self, user_input):
        self.messages.append({'role': 'user', 'content': user_input})
        answer = self.client.chat.completions.create(model=self.model, messages=self.messages)
        answer = answer.choices[0].message.content
        self.messages.append({'role': 'assistant', 'content': answer})
        return answer

In [2]:
system_rag = """
Ты ассистент оператора службы поддержки в компании Rutube.
Твоя задача - отвечать на запросы клиентов по 
использованию сервиса в службу поддержки.

Будь вежлив, обращайся к клиенту на Вы с большой буквы.

Информация из документа приведена ниже.
---------------------
{получен_чанк}
---------------------
Ответь на вопрос, основываясь на информации из документа.
Используй только фактические аднные из документа.
Теюе запрещено что-то придумывать и пытаться ответить без 
опоры на информацию из документа. 

Запрос: {вопрос}
Ответ:

"""
# Если информация в документе не отвечает на вопрос пользователя, верни None

In [3]:
system_validation = """
Ты являешься ассистентом оператора службы поддержки 
сервиса Rutube. Твоя задача - упростить для оператора процесс
поиск необходимой информации в документах для ответа на
вопрос пользователя.
Тебе нужно:
1. Изучить переданные документы
2. Определить, какая информация важна и полезна при ответе на вопрос пользователя
3. Вывести необходимую информцию оператору, чтобы он мог ответить на вопрос пользователя

Не обязательно должно быть строгое соответствие по смыслу или ключевым словам. 
Цель состоит в том, чтобы отфильтровать ошибочные факты,документы 
и извлечь только необходимую информацию для ответа на вопрос пользователя.
Изучи переданные документы и определи, какая информация из них нужна, чтобы точно,
и исчерпывающе ответить на вопрос пользователя.
В ответе выведи только ту информаци, которая будет полезна для ответа.

Информация из документов:
{relevant_docs}

Вопрос пользователя:
{user_query}

Информация из документов для ответа на вопрос:
"""